In [157]:
Pkg.add("MLDatasets")
Pkg.add("IJulia")
Pkg.add("StableRNGs")
Pkg.add("DataFrames")
Pkg.add("PyPlot")

ENV["PYTHON"] = ""

   Resolving package versions...

  No Changes to `C:\Users\parzy\.julia\environments\v1.7\Project.toml`
  No Changes to `C:\Users\parzy\.julia\environments\v1.7\Manifest.toml`


   Resolving package versions...


  No Changes to `C:\Users\parzy\.julia\environments\v1.7\Project.toml`
  No Changes to `C:\Users\parzy\.julia\environments\v1.7\Manifest.toml`

   Resolving package versions...


...

   Resolving package versions...


  No Changes to `C:\Users\parzy\.julia\environments\v1.7\Project.toml`
  No Changes to `C:\Users\parzy\.julia\environments\v1.7\Manifest.toml`


   Resolving package versions...


  No Changes to `C:\Users\parzy\.julia\environments\v1.7\Project.toml`
  No Changes to `C:\Users\parzy\.julia\environments\v1.7\Manifest.toml`


""

# Load Dataset

In [7]:
include("utils/GlobalUsings.jl")
include("utils/DataLoader.jl")
include("Lib/CustomDiffLib.jl")
include("utils/NetFunctions.jl")

X_train, y_train, X_test, y_test, classes  = getPrepearedData()

([-0.4275145876309958 -1.0397073145488935 … -1.284584405316053 1.2866250477391183; -1.2648671427817586 0.5749396103553435 … -0.11498792207106952 0.11498792207106849; 0.10064120238011458 -1.4017537805192064 … -1.4017537805192064 0.6207010041529566; 0.08842173269053696 -1.370536856703343 … -1.2379042576675354 0.3536869307621514], Bool[0 1 … 1 0; 1 0 … 0 1; 0 0 … 0 0], [1.7763792292734357 -0.4275145876309958 … -0.4275145876309958 -0.9172687691653144; -0.3449637662132075 2.6447222076345844 … 1.0348912986396195 1.7248188310660324; 1.4296829180218213 -1.4017537805192064 … -1.459538202938411 -1.2861849356807968; 0.7515847278695732 -1.370536856703343 … -1.370536856703343 -1.370536856703343], Bool[0 1 … 1 1; 0 0 … 0 0; 1 0 … 0 0], ["setosa", "versicolor", "virginica"])

### Working net

In [168]:
Random.seed!(2)
mean_squared_loss(y::BitVector, ŷ::Vector) = sum(0.5(y - ŷ) .^ 2)
fullyconnected(w::Vector, n::Number, m::Number, v::Vector, activation::Function) = activation.(reshape(w, n, m) * v)
σ(x) = one(x) / (one(x) + exp(-x))

function foward(x, wh, wo, y)
    x̂ = fullyconnected(wh, 4, 4, x, ReLU)
    ŷ = fullyconnected(wo, 3, 4, x̂, σ)
end

function net(x, wh, wo, y)
    x̂ = fullyconnected(wh, 4, 4, x, ReLU)
    ŷ = fullyconnected(wo, 3, 4, x̂, σ)
    E = mean_squared_loss(y, ŷ)
end


function train(epochs::Int, lr::Float64, X_data, y_data)
    X_data = X_train
    y_data = y_data
    Wh, Wo, dWh, dWo = InitializeNet()
    # Loss, accuracy tracking while net is training
    Loss_history = Float64[]
    Accuracy_history = Float64[]

    for epoch = 1:epochs
        epoch_L = []
        for i in 1:size(X_train)[2]
            x = X_train[:, i]
            y = y_data[:, i]
            L = net(x, Wh[:], Wo[:], y)
            dnet_Wh(x, wh, wo, y) = J(w -> net(x, w, wo, y), wh)
            dWh[:] = dnet_Wh(x, Wh[:], Wo[:], y)
            dnet_Wo(x, wh, wo, y) = J(w -> net(x, wh, w, y), wo)
            dWo[:] = dnet_Wo(x, Wh[:], Wo[:], y)
            push!(epoch_L, L)
            acc = foward(x, Wh[:], Wo[:], y)
            Wh -= lr * Wh
            Wo -= lr * dWo
        end
        push!(Loss_history, std(epoch_L))
    end
    return Loss_history,Accuracy_history
end

train (generic function with 1 method)